# Face Detection Using CNN Algorithms

>**This project is for building CNN architecture for facial features and face detection by plotting points on the facial features and box around the face**

In [ ]:
# import packages
import PIL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing import image

%matplotlib inline

## 1. Data Wrangling

### 1.1. Data Loading

>  
**Data Source:** https://www.kaggle.com/jessicali9530/celeba-dataset  
But we are going to **use only the data of the first 35000 images** as a start then we are going to update the model step by step beacuse the data is very very big **(1 GB)**

In [ ]:
# This cell for the values that will be used across the entire notebook
# pathes
key_points_data_path = "../input/celeba-dataset/list_landmarks_align_celeba.csv"
images_data_path = "../input/celeba-dataset/img_align_celeba/img_align_celeba"

# data size
images_data_size = 35000

# originsl image dimensions
x_org = 178    # original x value
y_org = 218    # original y value

# new image dimensions
x_ = 45                             # new value of x
image_size_ratio = x_org / y_org    # dimensions ratio
y_ = int(image_size_ratio * x_)     # new value of y

# image sizes
original_image_size = (x_org, y_org)
new_image_size = (x_,y_)

# the image size that will be used in the training process
image_size_training = new_image_size

# check the new size value
new_image_size

#### 1.1.1. Key points data

In [ ]:
# load the dataset (key points)
df_org = pd.read_csv(key_points_data_path)
df_org = df_org[:images_data_size]

# check
df_org.head(3)

In [ ]:
df_org.info()

#### 1.1.2. Images Data

In [ ]:
# convert images to arrays to use it in training
images_data = list()
for idx in range(df_org.shape[0]):
    # to get the path based on index
    path = "{}/{}".format(str(images_data_path),str(df_org.iloc[idx].image_id))
    
    # to read the image
    image = PIL.Image.open(path).resize(image_size_training)
    image_array = np.asarray(image) / 255
    
    # append the image array to images_data
    images_data.append(image_array)
    
# convert images_data to be array not list
images_data = np.array(images_data)

# check
images_data.shape

In [ ]:
# test
plt.imshow(images_data[18]);

In [ ]:
# Final data sets which we will work with
print("Images Data Arrays Shape:", images_data.shape)
print("Key Points Data Shape:", df_org.shape)

### 1.2. Data Cleaning (Preprocessing)

In [ ]:
# check if there is any null values in the data
df_org.isnull().sum()

In [ ]:
# check the numerical values properties in the data
df_org.describe()

> **Data is clean and ready for analysis**

### 1.3. Images reading and plotting

In [ ]:
# function to read images based on index
def image_array(index, size=image_size_training, path=images_data_path):
    """
    This functions is for converting images to arrays to deal with it in the model.
    
    Input:  index of the image that we want to convert to array
            size of the image that we want for the array of the image
            path of the images data to get the image
            
    Output: the image array as numpy array
    """
    # to get the path based on index
    path = "{}/{}".format(str(path),str(df_org.iloc[index].image_id))
    
    # to read the image
    image = PIL.Image.open(path).resize(size)
    image_array = np.asarray(image)
    
    return image_array

In [ ]:
# function to get a list of all key points of the face
def image_key_points_list(index, df = df_org):
    """
    This function for getting the key points on the face as list to deal with it in plotting sections
    """
    # box dictionary
    points_list = [df.iloc[index].lefteye_x,
                   df.iloc[index].lefteye_y,
                   df.iloc[index].righteye_x,
                   df.iloc[index].righteye_y,
                   df.iloc[index].nose_x,
                   df.iloc[index].nose_y,
                   df.iloc[index].leftmouth_x,
                   df.iloc[index].leftmouth_y,
                   df.iloc[index].rightmouth_x,
                   df.iloc[index].rightmouth_y]
    
    return points_list

In [ ]:
# function to plot the image with green box around the faces
def plotting_image_with_box(index, df = df_org, size=original_image_size):
    """
    This function for plotting the image with points on facial features and box around the face
    """
    test_image = image_array(index, size)
    points_list = image_key_points_list(index, df)
    
    # face points
    le_x, le_y, re_x, re_y = points_list[0], points_list[1], points_list[2], points_list[3]
    n_x, n_y = points_list[4], points_list[5]
    lm_x, lm_y, rm_x, rm_y = points_list[6], points_list[7], points_list[8], points_list[9]
    
    # Create figure and axes
    fig, ax = plt.subplots()
    # plot the image
    ax.imshow(test_image)
    # plot the points on the face
    ax.plot([le_x,re_x,n_x,lm_x,rm_x], [le_y,re_y,n_y,lm_y,rm_y], 'bo-')
    
    # plot the box around the face
    width = abs(le_x-rm_x-60)
    height = abs(le_y-rm_y-75)
    rect = patches.Rectangle((le_x-30, le_y-40), width, height, linewidth=4, edgecolor='g', facecolor='none')
    ax.add_patch(rect);

In [ ]:
# test
index = 18
plotting_image_with_box(index)

## 2. Preparing the data for ML model

### 2.1. Rescaling key points to be consistent with the new image size for training

In [ ]:
# copy a version from the data to prepare it for analysis
df = df_org.copy()

# check
df.head(3)

In [ ]:
# function for updating key points for a new size
def rescale_key_points(oldsize=original_image_size, newsize=image_size_training):
    """
    This function is for rescaling the key points from the original scale to a nwe scale from our chossen and we reduce
    the image size to make the analysis faster and using lower memory
    """
    # old and nwe sizes (x,y) values
    x_axis_old = oldsize[0]
    y_axis_old = oldsize[1]

    x_axis_new = newsize[0]
    y_axis_new = newsize[1]

    x_ratio = x_axis_new / x_axis_old
    y_ratio = y_axis_new / y_axis_old
    
    # converting the keypoints values to be trained with the new size of the images
    keypoints_x = ['lefteye_x', 'righteye_x', 'nose_x', 'leftmouth_x', 'rightmouth_x']
    keypoints_y = ['lefteye_y', 'righteye_y', 'nose_y', 'leftmouth_y', 'rightmouth_y']
    
    df[keypoints_x] = (df[keypoints_x] * x_ratio).astype('int')
    df[keypoints_y] = (df[keypoints_y] * y_ratio).astype('int')
    
    return 0

# call the function
rescale_key_points()

# check
df.head()

### 2.2. Split the data into training and test datasets

In [ ]:
# training data
train_labels = df[:images_data_size - 1000]
train_images = images_data[:images_data_size - 1000]


# test data (1000 sample)
test_labels = df[images_data_size - 1000 + 1:]
test_images = images_data[images_data_size - 1000 + 1:]

## 2.3 Split the data into train and validation datasets

In [ ]:
# droping image_id column as we will not use it in the model
y = train_labels.drop(['image_id'], axis = 1)      # labels
X = train_images                                   # images

# check
y.head(2)

In [ ]:
# to split the data into two sets (one for training and one for testing)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.17,random_state = 42)

# check the ratio
X_val.shape[0]/X_train.shape[0]

## 3. Building and training the model

In [ ]:
# diminsions of the image in the traing process
x_ = image_size_training[0]
y_ = image_size_training[1]

# build the model
model = Sequential()

model.add(Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation="relu", input_shape=(y_,x_,3)))
model.add(Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='relu'))

In [ ]:
# get the summary of the model layers
model.summary()

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
# fitting the model with our data
training_process = model.fit(X_train, y_train, epochs=200, validation_data=(X_val, y_val), batch_size=300, shuffle=True)

## 4. Test the model

In [ ]:
def predictions_test_model(index):
    img = tf.keras.preprocessing.image.load_img("{}/0{}.jpg".format(images_data_path, index),target_size=(y_,x_,3))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img/255
    points_list = model.predict(img.reshape(1,y_,x_,3)).astype('int')[0]
    
    # converting key points values to the original size
    x_ratio = 1.05 * (original_image_size[0] / image_size_training[0])
    y_ratio = 1.085 * (original_image_size[1] / image_size_training[1])
    """
    In the previous ratios we multiply them by contant to reduce the noise that happened when we rescaled the points in
    the previous training, there is no meaning for these numbers (i just pick them with trails)
    """
    
    points_list[0] = int(points_list[0] * x_ratio)
    points_list[2] = int(points_list[2] * x_ratio)
    points_list[4] = int(points_list[4] * x_ratio)
    points_list[6] = int(points_list[6] * x_ratio)
    points_list[8] = int(points_list[8] * x_ratio)
    
    points_list[1] = int(points_list[1] * y_ratio)
    points_list[3] = int(points_list[3] * y_ratio)
    points_list[5] = int(points_list[5] * y_ratio)
    points_list[7] = int(points_list[7] * y_ratio)
    points_list[9] = int(points_list[9] * y_ratio)
    
    return points_list

In [ ]:
# function to plot the image with green box around the faces
def test_image_with_box_plot(index, pred_or_actual = 'pred', pointsColor='bo-' ,boxcolor='g'):
    img = tf.keras.preprocessing.image.load_img("{}/0{}.jpg".format(images_data_path, index),target_size=(y_org,x_org,3))
    img = tf.keras.preprocessing.image.img_to_array(img)
    test_image = img/255
    
    # predictions of key points on the face
    if pred_or_actual == 'pred':
        points_list = predictions_test_model(index)        # this for predections of the model
    elif pred_or_actual == 'actual':
        points_list = image_key_points_list(index)   # this for the actual labels of the test data
    
    # face points
    le_x, le_y, re_x, re_y = points_list[0], points_list[1], points_list[2], points_list[3]
    n_x, n_y = points_list[4], points_list[5]
    lm_x, lm_y, rm_x, rm_y = points_list[6], points_list[7], points_list[8], points_list[9]

    # Create figure and axes
    fig, ax = plt.subplots()
    # plot the image
    ax.imshow(test_image)
    # plot the points on the face
    ax.plot([le_x,re_x,n_x,lm_x,rm_x], [le_y,re_y,n_y,lm_y,rm_y], pointsColor)
    
    # plot the box around the face
    width = abs(le_x-rm_x-60)
    height = abs(le_y-rm_y-75)
    rect = patches.Rectangle((le_x-30, le_y-40), width, height, linewidth=4, edgecolor=boxcolor, facecolor='none')
    ax.add_patch(rect);
    return points_list

In [ ]:
# test the model performace
index = 34100
print('RED box for predections\n') 
print('GREEN box for actual labels\n')
test_image_with_box_plot(index, pred_or_actual = 'pred', pointsColor='mo-' ,boxcolor='r')
test_image_with_box_plot(index, pred_or_actual = 'actual')

In [ ]:
# losses of both training and validation sets
loss = training_process.history['loss']
val_loss = training_process.history['val_loss']

# plot both losses
plt.plot(loss)
plt.plot(val_loss)
plt.legend(['loss', 'val_loss']);

In [ ]:
# accuracy of the model
accuracy = model.evaluate(X_val,y_val)[1] * 100
print("Accuracy of the model = ", round(accuracy,2))

## 5. Save the model

In [ ]:
# saving the model as h5 file
model.save('model.h5')